In [1]:
!git clone https://github.com/Mike030668/Project_glass.git -q

In [2]:
if 'google.colab' in str(get_ipython()):
  print('Running on CoLab')
  import os
  os.chdir('/content/Project_glass')
  os.getcwd()

else:
  print('Not running on CoLab')

Running on CoLab


In [16]:
from prepare_data.constants import FUTURE, GLASS_COLS, MAIN_COLS, MAKE_LOG
from prepare_data.utils import future_sequence, split_sequence, clean_dataset
import tensorflow as tf  # библиотека машинного обучения
import pandas as pd # библиотека пандас
import numpy as np # библиотека нампи
import joblib

## Готовим производные данных

In [5]:
all_data = pd.read_csv("/content/Project_glass/stock_data/BTCUSDT_1 _min.csv",
                       index_col=0, parse_dates=True )
print(f'Размер данных {all_data.shape}')

TEST_SIZE = 3500 # для теста откусим последние test_size значений
# Тестовая выборка для симуляции торговли в реальном времени
print(f'Тестовый датасет')
TEST_DATASET = all_data[FUTURE: FUTURE + TEST_SIZE]
print(TEST_DATASET.shape)


Размер данных (8317, 54)
Тестовый датасет
(3500, 54)


## Trend data

In [10]:
from prepare_data.conts_predtrend import  DEPTH, PREDICT_LAG
# колонки для тренировочной выборки
TRAIN_COLUMNS = MAIN_COLS + GLASS_COLS
GLASS = len(GLASS_COLS)

X_MAIN_SCALER = joblib.load("/content/Project_glass/predtrend/x_main_scailer.save")
X_GLASS_SCALER = joblib.load("/content/Project_glass/predtrend/x_glass_scailer.save")


x_test = TEST_DATASET[TRAIN_COLUMNS].values

x_trend = np.zeros_like(x_test)
x_trend[:,:-GLASS] = X_MAIN_SCALER.transform(x_test[:,:-GLASS])  # трансформируем x_train
x_trend[:,-GLASS:] = X_GLASS_SCALER.transform(x_test[:,-GLASS:])  # трансформируем x_train


print(x_trend.shape)
x_trend[0]

(3500, 54)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.3.0 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


array([0.49983619, 0.48335004, 0.53749873, 0.49514033, 0.03668795,
       0.7562355 , 0.02683915, 0.05958327, 0.29078355, 0.09084366,
       0.03806354, 0.76106195, 0.76415094, 0.23584906, 0.70063291,
       0.73325213, 0.26674787, 0.27573844, 0.47090281, 0.32923867,
       0.        , 0.        , 0.        , 0.019294  , 0.02835138,
       0.00593992, 0.03421547, 0.04401224, 0.01420412, 0.11746985,
       0.02197811, 0.02311966, 0.04296541, 0.08378829, 0.07066942,
       0.10173792, 0.06989426, 0.11310029, 0.03176935, 0.01433285,
       0.01492496, 0.04474071, 0.02132289, 0.05911877, 0.0851786 ,
       0.03115975, 0.0157034 , 0.20333738, 0.27896019, 0.54587476,
       0.22496303, 0.1756491 , 0.16825669, 0.25286763])

In [14]:
# Формирование выборок для обучения нейросети
x_trend = split_sequence(x_trend, DEPTH, PREDICT_LAG)
x_trend.shape

(3464, 32, 54)

In [15]:
trend_model =  tf.keras.models.load_model('./predtrend/model')

## Price data

In [ ]:
from prepare_data.const_predprice import  TARGET_COLS, BASE_COLS, BATCH_SIZE, DEPTH, PREDICT_LAG

# колонки для тренировочной выборки
TRAIN_COLUMNS = BASE_COLS + GLASS_COLS
GLASS = len(GLASS_COLS)

X_MAIN_SCALER = joblib.load("/content/Project_glass/predprice/x_main_scailer.save")
X_GLASS_SCALER = joblib.load("/content/Project_glass/predprice/x_glass_scailer.save")


x_test = TEST_DATASET[TRAIN_COLUMNS].values

x_price = np.zeros_like(x_test)
x_price[:,:-GLASS] = X_MAIN_SCALER.transform(x_test[:,:-GLASS])  # трансформируем x_train
x_price[:,-GLASS:] = X_GLASS_SCALER.transform(x_test[:,-GLASS:])  # трансформируем x_train


print(x_price.shape)
x_price[0]



In [17]:
def prepare_price_data(df_in: pd.DataFrame,
                        depth:int,
                        make_log,
                        scailers,
                        glass,
                        features
                      ):
    # обогащаем данные ранее созданной функцией
    ds = df_in.copy()
    # Удаление строк с неполными данными
    ds = clean_dataset(ds)
    # оставляем на глубину depth и только нужные колонки
    x_data = ds[features][-depth:].values
    # нормализуем
    x_out = np.zeros_like(x_data)
    x_out[:,:-glass] = scailers[0].transform(x_data[:,:-glass])  # трансформируем x_train
    x_out[:,-glass:] = scailers[1].transform(x_data[:,-glass:])  # трансформируем x_train

    # выводим с добавлением измерения по axis=0
    return np.expand_dims(x_out, axis=0)